In [33]:
import numpy as np
import pandas as pd
import os
import csv
import ntpath
import datetime
import operator
import time
import statistics as stats
import matplotlib.pyplot as plt
from collections import OrderedDict
from pathlib import Path
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

In [34]:
datasets = {}
files = []

def loadFile(path):
    global files
    global datasets
    fileExists = os.path.isfile(path)
    if not fileExists:
        return False
    files.append(path)
    filename,fileExtension = os.path.splitext(path)
    if '.csv' == fileExtension:
        df = pd.read_csv(path)
    elif '.xlsx' == fileExtension:
        df = pd.read_excel(path)
    else:
        return False
    datasets[ntpath.basename(path)] = df
    return True

loadFile("Features data set.csv")
loadFile("sales data-set.csv")
loadFile("stores data-set-withDMA.xlsx")

True

In [35]:
print("Features data set:")
print(datasets["Features data set.csv"].describe())
print("Dimensions: " + str(datasets["Features data set.csv"].shape[0]) + "x" + str(datasets["Features data set.csv"].shape[1]))
print()
print("sales data set:")
print(datasets["sales data-set.csv"].describe())
print("Dimensions: " + str(datasets["sales data-set.csv"].shape[0]) + "x" + str(datasets["sales data-set.csv"].shape[1]))
print()
print("stores data set (with DMA):")
print("Dimensions: " + str(datasets["stores data-set-withDMA.xlsx"].shape[0]) + "x" + str(datasets["stores data-set-withDMA.xlsx"].shape[1]))

Features data set:
             Store  Temperature   Fuel_Price      MarkDown1      MarkDown2  \
count  8190.000000  8190.000000  8190.000000    4032.000000    2921.000000   
mean     23.000000    59.356198     3.405992    7032.371786    3384.176594   
std      12.987966    18.678607     0.431337    9262.747448    8793.583016   
min       1.000000    -7.290000     2.472000   -2781.450000    -265.760000   
25%      12.000000    45.902500     3.041000    1577.532500      68.880000   
50%      23.000000    60.710000     3.513000    4743.580000     364.570000   
75%      34.000000    73.880000     3.743000    8923.310000    2153.350000   
max      45.000000   101.950000     4.468000  103184.980000  104519.540000   

           MarkDown3     MarkDown4      MarkDown5          CPI  Unemployment  
count    3613.000000   3464.000000    4050.000000  7605.000000   7605.000000  
mean     1760.100180   3292.935886    4132.216422   172.460809      7.826821  
std     11276.462208   6792.329861   1308

In [36]:
markDownColumnNames = []
for i in range(1,6):
    markDownColumnNames.append('MarkDown' + str(i))

datasets["Features data set.csv"] = datasets["Features data set.csv"].drop(markDownColumnNames,axis=1)

print("Features data set (updated)")
print(datasets["Features data set.csv"].describe())

Features data set (updated)
             Store  Temperature   Fuel_Price          CPI  Unemployment
count  8190.000000  8190.000000  8190.000000  7605.000000   7605.000000
mean     23.000000    59.356198     3.405992   172.460809      7.826821
std      12.987966    18.678607     0.431337    39.738346      1.877259
min       1.000000    -7.290000     2.472000   126.064000      3.684000
25%      12.000000    45.902500     3.041000   132.364839      6.634000
50%      23.000000    60.710000     3.513000   182.764003      7.806000
75%      34.000000    73.880000     3.743000   213.932412      8.567000
max      45.000000   101.950000     4.468000   228.976456     14.313000


In [37]:
#Get the row indicies at which there is a missing value
def getNAIndexes(data):
    i=0
    NAIndexes = []

    for row in pd.isna(data):
        if row == True:
            NAIndexes.append(i)
        i+=1
    return NAIndexes

#Given the missing row indicies, return a dictionary containing key:list(storeNumbers)
def getNAInfo(data,indexes):
    NAInfo = {}
    for index in indexes:
        day,month,year = data[index:index+1]["Date"].values[0].split('/')
        date = month + "/" + year
        store = data[index:index+1]["Store"].values[0]
        if date not in NAInfo.keys():
            NAInfo[date] = []
        if store not in NAInfo[date]:
            NAInfo[date].append(store)
    return NAInfo

#Get the length of a dictionary of lists
def lenDictOfLists(dictToCount):
    count = 0
    for listToCount in dictToCount.values():
        count += len(listToCount)
    return count

missingCPIInfo = getNAInfo(datasets["Features data set.csv"],getNAIndexes(datasets["Features data set.csv"]["CPI"]))
missingUnemploymentInfo = getNAInfo(datasets["Features data set.csv"],getNAIndexes(datasets["Features data set.csv"]["CPI"]))

print(missingCPIInfo)
print()
print(missingUnemploymentInfo)
print()
print("Are the rows missing a CPI value, also missing an Unemployment value? " + str(missingCPIInfo == missingUnemploymentInfo))
print("Amount of unique missing CPIs: " + str(lenDictOfLists(missingCPIInfo)))
print("Amount of unique missing Unemployment rates: " + str(lenDictOfLists(missingUnemploymentInfo)))
print("Total unique missing values: " + str((lenDictOfLists(missingCPIInfo) + lenDictOfLists(missingUnemploymentInfo))))

{'05/2013': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], '06/2013': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], '07/2013': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]}

{'05/2013': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], '06/2013': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], '07/2013': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19

In [46]:
monthNumberToMonthName = {
    1 : "January",
    2 : "February",
    3 : "March",
    4 : "April",
    5 : "May",
    6 : "June",
    7 : "July",
    8 : "August",
    9 : "September",
    10 : "October",
    11 : "November",
    12 : "December"
}

def getColumnIndex(dataFrame,searchTerm,isExactMatch = False): #Return column index number given column name in dataFrame
    i=-1
    for columnName in list(dataFrame):
        if(isExactMatch):
            condition = searchTerm == columnName
        else:
            condition = searchTerm in columnName
        i+=1
        if condition == True:
            return i

def buildMissingCPIValues(debug=False): # Build a dictionary containing all missing CPI values from external datasets
    global datasets
    global missingCPIInfo
    missingValues = {}
    missingValueErrors = []
    success = True
    for date,stores in missingCPIInfo.items():
        for store in stores:
            month,year = date.split('/')
            month = "M" + str(int(month)).zfill(2)
            store = int(store)
            targetStoreRow = datasets["stores data-set-withDMA.xlsx"].query("Store == @store")
            targetDMA = targetStoreRow.values[0][getColumnIndex(targetStoreRow,"DMA")]
            targetCPIMapRow = datasets["custom-dma-to-cpiArea.csv"].query("DMA == @targetDMA")
            targetCPIArea = targetCPIMapRow.values[0][getColumnIndex(targetCPIMapRow,"CPI_AreaCode")]
            missingValueFulfilled = False
            firstAttemptRan = False
            if(type(targetCPIArea) == float): #DMA has missing values, in which case pull from missing values csv
                firstAttemptRan = True
                cpiRow = datasets["custom-dma-to-cpiMissingValuesYearly.csv"].query("DMA == @targetDMA and year == @year")
                if len(cpiRow) > 0:
                    cpi = cpiRow.values[0][getColumnIndex(cpiRow,"value")]
                    missingValueFulfilled = True
            if not missingValueFulfilled and not firstAttemptRan:
                cpiRowsAreaFiltered = datasets["cu.data.0.Current.csv"][datasets["cu.data.0.Current.csv"]['series_id'].str.contains(targetCPIArea) == True]
                cpiRowsDateFiltered = cpiRowsAreaFiltered.query("year == @year & period == @month")
                if(len(cpiRowsDateFiltered) <= 0): # Dates have missing values, in which case pull from missing values CSV
                    cpiRow = datasets["custom-dma-to-cpiMissingValuesYearly.csv"].query("DMA == @targetDMA and year == @year")
                    if len(cpiRow) > 0:
                        cpi = cpiRow.values[0][getColumnIndex(cpiRow,"value")]
                        missingValueFulfilled = True
                else: #average all CPI values with the area code
                    count = 0
                    sum = 0
                    for index,row in cpiRowsDateFiltered.iterrows():
                        sum += row['value']
                        count+=1
                    mean = sum/count
                    cpi = mean
                    missingValueFulfilled = True
            if not missingValueFulfilled:
                    missingValueErrors.append([store,date])
                    success = False
            else:        
                missingValues[(store,date)] = cpi   
    if(debug and not success):
        print("ERROR: Could not build full CPI dictionary")
        return missingValueErrors
    else:
        if(not success):
            return success
        else:
            return missingValues
def buildMissingUnemploymentRates(debug=False): # Build a dictionary containing all missing unemployment values from external datasets
    global datasets
    global missingUnemploymentInfo
    missingValues = {}
    missingValueErrors = []
    success = True
    for date,stores in missingUnemploymentInfo.items():
        for store in stores:
            month,year = date.split('/')
            month = monthNumberToMonthName[int(month)]
            store = int(store)
            targetStoreRow = datasets["stores data-set-withDMA.xlsx"].query("Store == @store")
            targetDMA = targetStoreRow.values[0][getColumnIndex(targetStoreRow,"DMA")]
            
            targetUnemploymentMapRow = datasets["custom-dma-to-unemploymentCounty.csv"].query("DMA == @targetDMA")
            targetState = targetUnemploymentMapRow.values[0][getColumnIndex(targetUnemploymentMapRow,"UnemploymentState")]
            targetCounty = targetUnemploymentMapRow.values[0][getColumnIndex(targetUnemploymentMapRow,"UnemploymentCounty")]
            missingValueFulfilled = False
            firstAttemptRan = False
            if type(targetCounty) == float: # County has missing values, in which case pull from missing values csv
                firstAttemptRan = True
                unemploymentRateRow = datasets["custom-dma-to-unemploymentMissingValues.csv"].query("DMA == @targetDMA and Year == @year and Month == @month")
                if len(unemploymentRateRow) > 0:
                    unemploymentRate = unemploymentRateRow.values[0][getColumnIndex(unemploymentRateRow,"UnemploymentRate")]
                    missingValueFulfilled = True
            if not missingValueFulfilled and not firstAttemptRan:
                unemploymentRateRow = datasets["output.csv"].query("Year == @year & Month == @month & State == @targetState & County == @targetCounty")
                if(len(unemploymentRateRow > 0)):
                    unemploymentRate = unemploymentRateRow.values[0][getColumnIndex(unemploymentRateRow,"Rate")]
                    missingValueFulfilled = True
                else: # Missing values for the dates in the dataset, in which case pull from missing values csv
                    unemploymentRateRow = datasets["custom-dma-to-unemploymentMissingValues.csv"].query("DMA == @targetDMA and Year == @year and Month == @month")
                    if len(unemploymentRateRow) > 0:
                        unemploymentRate = unemploymentRateRow.values[0][getColumnIndex(unemploymentRateRow,"UnemploymentRate")]
                        missingValueFulfilled = True
            if not missingValueFulfilled:
                missingValueErrors.append([store,date])
                success = False
            else:        
                missingValues[(store,date)] = unemploymentRate
    if(debug and not success):
        print("ERROR: Could not build full unemployment dictionary")
        return missingValueErrors
    else:
        if(not success):
            return success
        else:
            return missingValues
        
def fillInFeaturesMissingValues(missingUnemploymentValues,missingCPIValues): # Fill in missing values on Features dataset given dictionaries generated from the above
    global datasets
    allValuesFilled = True
    #Basic error checking to make sure both arguments are in the correct format
    if(type(missingUnemploymentValues) != dict or type(missingCPIValues) != dict):
        return False
    for index,row in datasets["Features data set.csv"].iterrows():
        day,month,year = row['Date'].split('/')
        date = month + '/' + year
        store = int(row['Store'])   
        if(pd.isnull(row['Unemployment'])):
            if((store,date) in missingUnemploymentValues):
                datasets["Features data set.csv"].set_value(index,'Unemployment',missingUnemploymentValues[(store,date)])
            else:
                allValuesFilled = False    
        if(pd.isnull(row['CPI'])):
            if((store,date) in missingCPIValues):
                datasets["Features data set.csv"].set_value(index,'CPI',missingCPIValues[(store,date)])
            else:
                allValuesFilled = False
    return allValuesFilled

loadFile("output.csv")
loadFile("custom-dma-to-unemploymentCounty.csv")
loadFile("custom-dma-to-cpiArea.csv")
loadFile("custom-dma-to-unemploymentMissingValues.csv")
loadFile("custom-dma-to-cpiMissingValuesYearly.csv")
loadFile("cu.data.0.Current.csv")

print("Building dictionary of missing values to fill in...")
missingUnemploymentValues = buildMissingUnemploymentRates()
missingCPIValues = buildMissingCPIValues()
    
if(type(missingCPIValues) == dict and type(missingUnemploymentValues) == dict):
    print("Success")
    print("Filling in missing values in Features data set...")
    print("NOTE: This will only modify the dataset in memory, not the actual CSV file")
    fillInFeaturesMissingValues(missingUnemploymentValues,missingCPIValues)
    if len(datasets["Features data set.csv"].index) == datasets["Features data set.csv"].describe()['Unemployment']['count'] and len(datasets["Features data set.csv"].index) == datasets["Features data set.csv"].describe()['CPI']['count']:
        print("All CPI and Unemployment missing values have been filled")
    else:
        print("NOTE: Features dataset still has missing values")
else:
    print("One of the dictionary builds failed, exiting")

#Export modified Features dataset to file
exportPath = 'modified/Features data set.csv'
datasets["Features data set.csv"].to_csv(exportPath,index=False)
print("File exported to " + exportPath)

Building dictionary of missing values to fill in...
One of the dictionary builds failed, exiting
File exported to modified/Features data set.csv


In [47]:
def adjustFeaturesTimeline(startDate,endDate):
    global datasets
    startDate = datetime.datetime.strptime(startDate, "%d/%m/%Y").date()
    endDate = datetime.datetime.strptime(endDate, "%d/%m/%Y").date()
    for index,row in datasets["Features data set.csv"].iterrows():
        currentDate = datetime.datetime.strptime(row['Date'], "%d/%m/%Y").date()
        if currentDate > endDate or currentDate < startDate:
            datasets["Features data set.csv"] = datasets["Features data set.csv"].drop([index])

adjustFeaturesTimeline("05/02/2010","01/11/2012")

In [48]:
usefulData = [
    'Item',
    'Number of consumer units (in thousands)',
    'Income before taxes',
    'Age of reference person',
    'Adults 65 and older',
    'Earners',
    'Vehicles',
    'Percent homeowner',
    'Average annual expenditures',
    'Food',
    'Alcoholic beverages',
    'Apparel and services',
    'Personal care products and services',
    'Tobacco products and smoking supplies'
]

areas = {}
areaInfo = {}

loadFile("norteast.xlsx")
loadFile("midwest.xlsx")
loadFile("south.xlsx")
loadFile("west.xlsx")

C:\Users\prime\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\prime\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\prime\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\prime\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl'

True

In [49]:
def loadMSA(fileName):
    global datasets
    global usefulData
    global areas
    
    bufferListOfPlaces = []
    bufferDict = {}
    for value in datasets[fileName].values:
        attribute = value[0]
        if attribute in usefulData:
            if(attribute == 'Item'): # List of DMAs
                bufferListOfPlaces = value[1:]
            else:
                bufferDict[attribute] = value
    if len(bufferListOfPlaces) > 0:
        i = 1
        for place in bufferListOfPlaces:
            place = place.replace('\n',' ')
            for _,value in bufferDict.items():
                attribute = value[0]
                attributeValue = value[i]
                if place not in areas:
                    areas[place] = {}
                areas[place][attribute] = attributeValue
            i+=1
    else:
        return False
                
loadMSA("midwest.xlsx")
loadMSA("norteast.xlsx")
loadMSA("south.xlsx")
loadMSA("west.xlsx")

In [58]:
loadFile("custom-dma-to-consumerArea.csv")

sales = datasets["sales data-set.csv"].drop(['Dept','Date','IsHoliday'],axis=1)

def getSalesByStore():
    global sales
    storeSales = {}
    for _,row in sales.iterrows():
        storeNumber = int(row['Store'])
        weeklySales = row['Weekly_Sales']
        if storeNumber in storeSales:
            storeSales[storeNumber].append(weeklySales)
        else:
            storeSales[storeNumber] = [weeklySales]
    return storeSales

storeSales = getSalesByStore() # Very slow
DMASpendingPerCapita = {}
DMASpending = {}

for index,store in storeSales.items():
    DMA = mapStoreNumberToDMA(index)
    consumerInfoName = mapDMAToName(datasets["custom-dma-to-consumerArea.csv"],DMA)
    if type(consumerInfoName) != float:
        medianStoreSales = stats.median(storeSales[index])
        if DMA not in DMASpendingPerCapita:
            DMASpendingPerCapita[DMA] = []
            DMASpending[DMA] = []
        DMASpendingPerCapita[DMA].append(medianStoreSales)
for index,listOfSales in DMASpendingPerCapita.items():
    sum = 0
    for sale in listOfSales:
        sum+=sale
    consumerInfoName = mapDMAToName(datasets["custom-dma-to-consumerArea.csv"],index)
    DMASpending[index] = sum
    DMASpendingPerCapita[index] = sum/areas[consumerInfoName]['Number of consumer units (in thousands)']

print("Rankings of Overall Spending (Sum of Medians of Stores)")
for index,value in rankDictionary(DMASpending).items():
    print(index + " : " + str(value))
print()
print("Rankings of Spending per Capita (Sum of Medians of Stores/Number of Customer Units)")
for index,value in rankDictionary(DMASpendingPerCapita).items():
    print(index + " : " + str(value))

NameError: name 'mapStoreNumberToDMA' is not defined

In [63]:
clustersToTry = [2,3,4]

featuresSetForAnalysis = datasets["Features data set.csv"].drop(['IsHoliday','Date'],axis=1)

def runKMeans(data,numOfClusters):
    kmeans = KMeans(n_clusters=numOfClusters)
    kmeans.fit(data)
    y_km = kmeans.fit_predict(data)
    return y_km

for clusterNum in clustersToTry:
    exportPath = 'export\Features data set' + str(clusterNum) + '.csv'
    datasetToExport = datasets["Features data set.csv"].assign(cluster = runKMeans(featuresSetForAnalysis,clusterNum).tolist())
    datasetToExport.to_csv(exportPath,index=False)